# Ground path simulation

Simulates ground path of a satellite in two reference frames:
- [GCS](https://en.wikipedia.org/wiki/Geographic_coordinate_system) (geographic coordinate system, lat/lon/elevation)
- [ECI](https://en.wikipedia.org/wiki/Earth-centered_inertial) (Earth-centered intertial, X/Y/Z) reference frames.

In [1]:
import os
from skyfield.api import load
from ground_path import ground_path
import numpy as np
from datetime import datetime

### Load existing satellite from NORAD database

In [2]:
resource_url = 'http://celestrak.com/satcat/tle.php?CATNR=40299'

fname = 'tle_hodoyoshi_1.txt'
satellites = load.tle(resource_url, filename=fname, reload=False)

satellite = satellites['HODOYOSHI 1']
print(satellite)

EarthSatellite 'HODOYOSHI 1' number=40299 epoch=2018-10-24T12:03:08Z


### Calculate ground path

In [3]:
date_from = datetime(2018,8,1,0,0,0,0)
date_to = datetime(2018,8,2,0,0,0,0)
freq = 30 # generate data point every 30 seconds
df = ground_path(satellite, date_from, date_to, freq)
df.head()

,X,Y,Z,lat,lon,elevation,ascending,orbit
datetime,,,,,,,,
2018-08-01 00:00:00+00:00,554.203355,-3178.014692,6054.771852,62.118186,-29.613950,499.094947,True,0
2018-08-01 00:00:30+00:00,459.746660,-2995.194778,6154.700398,63.944699,-30.921147,499.336186,True,0
2018-08-01 00:01:00+00:00,364.781322,-2809.061221,6247.800731,65.759775,-32.392397,499.569012,True,0
2018-08-01 00:01:30+00:00,269.412421,-2619.820087,6333.969878,67.560777,-34.066235,499.791700,True,0
2018-08-01 00:02:00+00:00,173.745475,-2427.680857,6413.112555,69.344170,-35.993558,500.002625,True,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2880 entries, 2018-08-01 00:00:00+00:00 to 2018-08-01 23:59:30+00:00
Data columns (total 8 columns):
X            2880 non-null float64
Y            2880 non-null float64
Z            2880 non-null float64
lat          2880 non-null float64
lon          2880 non-null float64
elevation    2880 non-null float64
ascending    2880 non-null bool
orbit        2880 non-null int64
dtypes: bool(1), float64(6), int64(1)
memory usage: 182.8 KB


In [5]:
# save as CSV file
df.to_csv('ground_path_hodoyoshi.csv')

## Statistics

In [6]:
# Total distance approximation
pos = df[['X', 'Y', 'Z']]
dist = np.linalg.norm((pos - pos.shift()).dropna(), axis=1)
dist.sum()

657346.5005044239

In [7]:
df[['elevation', 'lon', 'lat']].agg(['min', 'max'])

,elevation,lon,lat
min,495.165387,-179.865451,-82.669256
max,534.512578,179.997911,82.669739


In [8]:
df[['elevation']].describe()

,elevation
count,2880.000000
mean,508.811566
std,13.340070
min,495.165387
25%,498.136634
50%,501.170742
75%,520.436724
max,534.512578
